### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### 1
The objective function is following:<br />
$min_{A_{12},A_{21}}\Sigma_{i=1}^{n}(p(x^{(i), A_{12}, A_{21}})-p^{(i)})^2$ <br />
where $n = 11$,  $x_2=1-x_1$<br />
$p(x^{(i)}, A_{12},A_{21})=x_1^{(i)}exp(A_{12}(\frac{A_{21}x_2^{(i)}}{A_{12}x^{(i)}_1+A_{21}x_2^{(i)}})^2)p_{water}^{sat}+x_2^{(i)}exp(A_{12}(\frac{A_{21}x_1^{(i)}}{A_{12}x^{(i)}_1+A_{21}x_2^{(i)}})^2)p_{1,4dioxane}^{sat}$

### 2

In [5]:
import torch as t
from torch.autograd import Variable
import numpy as np

pwater_sat = np.power(10, (8.071 - 1730.63/(20 + 233.426)))
p14_sat = np.power(10, (7.432 - 1554.68/(20 + 240.337)))
x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
p = [28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]

def pressure(a, xi):

    x1 = xi
    x2 = 1 - x1
    A12 = a[0]
    A21 = a[1]
    p_n = x1 * t.exp(A12* ( (A21*x2)/(A12*x1 + A21*x2) )**2 ) * pwater_sat + x2 * t.exp(A21* ( (A12*x1)/(A12*x1 + A21*x2) )**2 ) * p14_sat
    return p_n

def object_fun(a):

    loss_ = 0
    n = len(x)

    for i in range(n):
        xi = x[i]
        p_n = pressure(a, xi)
        loss_ = loss_ + (p_n - p[i])**2

    return loss_

def linesearch(a):

    step = 0.1
    while object_fun(a - step*a.grad) > object_fun(a)-step*(0.5)*np.matmul(a.grad, a.grad):
        step = 0.5 * step

    return step

a = Variable(t.tensor([1.0, 1.0]), requires_grad=True)
diff = 100

while diff > 0.1:

    obj = object_fun(a)
    obj.backward()
    step = linesearch(a)
    diff = t.linalg.norm(a.grad)

    with t.no_grad():
        a -= step * a.grad
        a.grad.zero_()

print('A12 and A21 is {}, respectly, with loss {}'.format(a.data.numpy(), obj.data.numpy()))

A12 and A21 is [1.9545084 1.6900573], respectly, with loss 0.7166754603385925


### 3

In [6]:
a_ = a.data
print('True Pressures   Model Pressures')
y_pred = np.zeros([len(x), 1])
for i in range(len(x)):
    y_pred[i] = pressure(a_, x[i])
    print('The true pressure is {}, and the model pressure is {}'.format(p[i], pressure(a, x[i])))

from sklearn.metrics import mean_squared_error
err = mean_squared_error(p, y_pred)
print('The mean squared error of the predicted data w.r.t true values is {}'.format(err))

True Pressures   Model Pressures
The true pressure is 28.1, and the model pressure is 28.85372543334961
The true pressure is 34.4, and the model pressure is 34.645721435546875
The true pressure is 36.7, and the model pressure is 36.45185852050781
The true pressure is 36.9, and the model pressure is 36.866844177246094
The true pressure is 36.8, and the model pressure is 36.873008728027344
The true pressure is 36.7, and the model pressure is 36.747642517089844
The true pressure is 36.5, and the model pressure is 36.38789367675781
The true pressure is 35.4, and the model pressure is 35.38384246826172
The true pressure is 32.9, and the model pressure is 32.949913024902344
The true pressure is 27.7, and the model pressure is 27.73257064819336
The true pressure is 17.5, and the model pressure is 17.460784912109375
The mean squared error of the predicted data w.r.t true values is 0.06515213753043846


Since the means squared error is 0.065, the model-predicted pressures fit very well with the data.

### Problem 2 (50 points)
Solve the following problem using Bayesian Optimization:
$$\min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).

In [1]:
import numpy as np
from sklearn import gaussian_process as gp
from scipy.stats import norm
from scipy.optimize import minimize

def object_fun(x):
    v = ((4 - (2.1*(x[0]**2)) + ((x[0]**4)/3))*(x[0]**2)) + (x[0]*x[1]) + ((-4 + 4*x[1]**2)*x[1]**2)
    return v

def expected_improvement(x, model, evaluated_loss, greater_is_better=False, n_params=1):

    x_to_predict = x.reshape(-1, n_params)
    mu, sigma = model.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0

    return -1 * expected_improvement


def sample_next_hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):

    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))

        if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x


def bayesian_optimisation(n_iters, sample_loss, bounds, x0=None, n_pre_samples=5,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7):

    x_list = []
    y_list = []

    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(sample_loss(params))
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(sample_loss(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for n in range(n_iters):

        model.fit(xp, yp)

        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * expected_improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = sample_next_hyperparameter(expected_improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        cv_score = sample_loss(next_sample)

        x_list.append(next_sample)
        y_list.append(cv_score)

        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

In [13]:
import warnings
warnings.filterwarnings('ignore')

x_1 = np.linspace(-3,3)
x_2 = np.linspace(-2,2)
param_grid = np.array([[x_1i, x_2i] for x_1i in x_1 for x_2i in x_2])
real_loss = [object_fun(params) for params in param_grid]
print('The minimum value of {} is at {} '.format(np.amin(real_loss), param_grid[np.array(real_loss).argmin(), :]))

bounds = np.array([[-3, 3], [-2, 2]])
xp, yp = bayesian_optimisation(n_iters=100,
                               sample_loss=object_fun,
                               bounds=bounds,
                               n_pre_samples=3,
                               random_search=100000)

print('For Bayesian Optimization with 100 iteration, the minimum value of {} is at {}'.format(np.amin(yp), xp[np.where(yp == np.amin(yp))[0][0]]))

The minimum value of -1.02614400718987 is at [-0.06122449  0.69387755] 
For Bayesian Optimization with 100 iteration, the minimum value of -1.002744475669542 is at [-0.14689115  0.67015273]
